## आवश्यक लाइब्रेरीहरू आयात गर्नुहोस्  
Phi-4 मोडेल लोड गर्न र प्रयोग गर्न आवश्यक PyTorch र transformers लाइब्रेरीहरू आयात गर्नुहोस्।  


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## र्यान्डम बीज सेट गर्नुहोस्  
विभिन्न रनहरूमा पुनःउत्पादनयोग्य नतिजा सुनिश्चित गर्न र्यान्डम बीज सेट गर्नुहोस्।


In [ ]:
torch.random.manual_seed(0)

## Phi-4-mini-flash-reasoning मोडेल र टोकनाइजर लोड गर्नुहोस्  
Microsoft को Phi-4-mini-flash-reasoning मोडेल र यसको सम्बन्धित टोकनाइजर Hugging Face बाट लोड गर्नुहोस्। मोडेललाई छिटो निष्कर्षका लागि CUDA मा लोड गरिनेछ।  


In [ ]:
model_id = "microsoft/Phi-4-mini-flash-reasoning"
model = AutoModelForCausalLM.from_pretrained(
   model_id,
   device_map="cuda",
   torch_dtype="auto",
   trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

## इनपुट सन्देश तयार गर्नुहोस्  
मोडेलको लागि च्याट टेम्प्लेट प्रयोग गरेर एक द्विघात समीकरण गणित समस्या सहितको कुराकानी सन्देश तयार गर्नुहोस्।


In [ ]:
messages = [{
   "role": "user",
   "content": "How to solve 3*x^2+4*x+5=1?"
}]   
inputs = tokenizer.apply_chat_template(
   messages,
   add_generation_prompt=True,
   return_dict=True,
   return_tensors="pt",
)

## प्रतिक्रिया उत्पन्न गर्नुहोस्  
मोडेलबाट प्रतिक्रिया उत्पन्न गर्नुहोस्, जस्तै तापक्रम र top_p जस्ता निर्दिष्ट प्यारामिटरहरू प्रयोग गरेर उत्पादनमा नियन्त्रणित अनियमितता सुनिश्चित गर्नुहोस्।


In [ ]:
outputs = model.generate(
   **inputs.to(model.device),
   max_new_tokens=32768,
   temperature=0.6,
   top_p=0.95,
   do_sample=True,
)

## आउटपुटलाई पाठमा डिकोड गर्नुहोस्
जोडिएका टोकन अनुक्रमलाई मानवीय रूपमा पढ्न मिल्ने पाठमा परिवर्तन गर्नुहोस्, मूल इनपुट टोकनहरूलाई हटाएर केवल मोडेलको प्रतिक्रिया देखाउनुहोस्।


In [ ]:
outputs = tokenizer.batch_decode(outputs[:, inputs["input_ids"].shape[-1]:])


---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरेर अनुवाद गरिएको छ। हामी शुद्धताको लागि प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छ। यसको मूल भाषा मा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
